In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import sys
import time
from sklearn.metrics import accuracy_score
from tqdm import tqdm_notebook as tqdm

sys.path.insert(0, "../../")

from rcalgo.tf.models.semi_supervised.adversarial_training import AdversarialTextClassWrapper, \
    AdversarialTraining
from rcalgo.tf.models.semi_supervised.data import SemiSupTrainData
from rcalgo.tf.models.classifier.text_classifier import TextCNN
from rcalgo.tf.utils.tokenizers import CharTokenizer
from rcalgo.tf.utils.serving_utils import add_prefix_to_ckpt
from rcalgo.tf.training.hook import TensorInfoPrintHook
from rcalgo.tf.training.data.data_iterator import DataIterator

In [2]:
# load data
data_dir = "/home/web_server/antispam/project/datasets/female_vulgar/"
train_data = pd.read_csv(data_dir + 'train_1w.csv')
test_data = pd.read_csv(data_dir + 'test.csv')
# count word dict
tokenizer = CharTokenizer.build_from_corpus(test_data['text'], freq_threshold=2)
word_dict = tokenizer.word2idx

In [3]:
org_unsup_file = data_dir + "unsup0.txt"

In [4]:
sup_texts = train_data['text']
labels = train_data['label']
batch_size = 32
unsup_ratio = 6
gpu_num = 2
num_epochs = 150
num_train_steps = num_epochs * (int(len(sup_texts)) // (batch_size * gpu_num))
print(f"num_train_steps: {num_train_steps}")

num_train_steps: 23400


In [5]:
# create execute model
config = {
    # uda config
    "unsup_coeff": 1,  # unsup loss coeff 
    "tsa_schedule": "linear_schedule",  # linear_schedule, exp_schedule, log_schedule, None
    "batch_size": batch_size,
    "unsup_ratio": unsup_ratio,  # unsup data ratio
    "num_train_steps": num_train_steps,  # num of train steps is needed for tas threshold calcuation
    "use_adv_train": False,
    "epsilon": 10,
    "xi": 0.1,
    "rampup_method": "sigmoid", 
    
    # model config
    "learning_rate": 0.001,

    "num_classes": 2,
    "max_seq_len": 100,
    "embedding_size": 128,
    "num_layers": 2,
    "conv_num_filters": 256,
    "keep_prob": 0.5,
    
    "gpu_id": 0,
    "gpu_num": gpu_num,

    "model_output_dir": "./saved_models",
    "model_name": "cnn"
}

In [6]:
test_sup_texts = test_data['text']
test_labels = test_data['label']

In [7]:
org_unsup_texts = []
with open(org_unsup_file) as org_f:
    all_org_unsup_texts = [line.strip() for line in org_f.readlines()]

unsup_size = len(all_org_unsup_texts) // unsup_ratio
for i in range(unsup_size):
    org_unsup_texts.append([all_org_unsup_texts[i * unsup_ratio + j] for j in range(unsup_ratio)])
    
test_size = len(test_labels)
test_org_unsup_texts = org_unsup_texts[:test_size]

In [8]:
print(f"sup_texts: ")
print(sup_texts[:5])
print(f"org_unsup_texts: ")
print(org_unsup_texts[:5])
print(len(org_unsup_texts))

sup_texts: 
0                上去不，宝贝
1            服务区这个小嘴太美了
2                 第二个雏吗
3       不要彩礼也不见得老公疼你一辈子
4    有没有发现大多都是女孩子患有抑郁症💔
Name: text, dtype: object
org_unsup_texts: 
[['祝雅儿新年快乐，开心快乐每一天，越来越漂亮哈[笑哭]', '非洲哥', '鬼街在哪啊', '哈卵', '差那点洗脸洗手的时间？', '?????😻😔🤒👏👨\u200d'], ['别动我要给你舔', '爱死宝贝师虎', '兜兜转转还是你🤘', '改革开放四十周年，吃不完穿不完的。真好！', '今天我湿答答的从你身边经过', '纹身吗😂'], ['那就祝贺你余生灌满浆，走路拉汤汤，', '快来找我我要馋死了[大哭]', '只有我看见四哥脖子上的草莓了么', '吟诗一首，黑草丛中一刀伤，不用浇水自带汤。两片红唇分大小，一根黄瓜塞满缸。站立樱花闭，蹲时牡丹开，动时双峰舞，用时水成灾，[偷笑]', '我以为是带了个尿不湿呢', '够玩够用'], ['这个姿势有奖励', '老婆让谁给货货大了', '你可还记得我啊？', '🐔吧样', '末将请求一战，如不出水，臣提头来见😀😂😂', '别人怎么说你的'], ['顺风车欢迎你', '希望你能从校服到婚纱', '要是叫的声音销魂我也就三四分，要是不喜欢能半个小时左右', '我会爱你到最后一刻', '山高地肥，费力费牛', '小妹妹好肥啊']]
66666


## training with original model

In [10]:
tf.compat.v1.reset_default_graph()

model = TextCNN(config, word_dict, name="default", distribute="tf_distribute")
model.build_model()
print(model.model_summary())

2020-11-11 18:27:39,231  model.py (73) (MainThread) : INFO  config: {'unsup_coeff': 1, 'tsa_schedule': None, 'batch_size': 32, 'unsup_ratio': 6, 'num_train_steps': 23200, 'use_adv_train': False, 'epsilon': 8, 'xi': 0.1, 'learning_rate': 0.001, 'num_classes': 2, 'max_seq_len': 100, 'embedding_size': 128, 'num_layers': 2, 'conv_num_filters': 256, 'keep_prob': 0.4, 'gpu_id': 0, 'gpu_num': 2, 'model_output_dir': './saved_models', 'model_name': 'cnn'}


Instructions for updating:
reduction_indices is deprecated, use axis instead
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

total parameter number: 581378
                         variable_name variable_shape parameters
0    embedding_layer/embedding_table:0    [3765, 128]     481920
1  my_conv_no.1/conv_1d/he_uniform/W:0  [1, 128, 128]      16384
2             my_conv_no.1/conv_1d/B:0          [128]        128
3  my_conv_no.2/conv_1d/he_uniform/W:0  [2, 128, 128]      32768
4             my_conv_no.2/conv_1d/B:0          [128]        128
5  my_conv_no.3/conv_1d/he_uniform/W:0  [3, 128, 128]      49152
6             my_conv_no.3/conv_1d/B:0          [128]        128
7                  fully_connected/W:0       [384, 2]        768
8                  fully_connected/B:0            [2]          2


In [11]:
model.train_and_evaluate([sup_texts, labels], [test_sup_texts, test_labels], 
            batch_size=batch_size * config['gpu_num'],
            epochs=30,
            hooks=[])

2020-11-09 18:36:48,603  single_worker_graph_copy_trainer.py (92) (MainThread) : INFO  Initializing trainer ...
2020-11-09 18:36:48,604  single_worker_graph_copy_trainer.py (105) (MainThread) : INFO  gpu_num: 2
2020-11-09 18:36:48,605  graph_copy.py (424) (MainThread) : INFO  build_train_graph kwargs: {'unsup_coeff': 1, 'tsa_schedule': 'linear_schedule', 'batch_size': 32, 'unsup_ratio': 6, 'num_train_steps': 34120, 'use_adv_train': True, 'epsilon': 5, 'xi': 0.01, 'learning_rate': 0.001, 'num_classes': 2, 'max_seq_len': 100, 'embedding_size': 128, 'num_layers': 2, 'conv_num_filters': 256, 'keep_prob': 0.5, 'model_output_dir': './saved_models', 'model_name': 'cnn', 'excluded_task_set': ['pretrained_model/default'], 'pretrained_model_scope': 'pretrained_model'}
2020-11-09 18:36:48,606  graph_copy.py (426) (MainThread) : INFO  build_train_graph: reconstructing variables...



Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2020-11-09 18:36:50,959  graph_copy.py (448) (MainThread) : INFO  build_train_graph (default): building inputs...
2020-11-09 18:36:50,960  graph_copy.py (450) (MainThread) : INFO  task: default, <class 'str'>
2020-11-09 18:36:50,964  graph_copy.py (478) (MainThread) : INFO  build_train_graph (default): building per replica graph...



INFO:tensorflow:batch_all_reduce: 9 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10


2020-11-09 18:36:51,981  graph_copy.py (486) (MainThread) : INFO  train_ops: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Adam_default/Identity:0' shape=() dtype=int64>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'Adam_default/Identity_1:0' shape=() dtype=int64>
}], losses: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Mean:0' shape=() dtype=float32>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'replica_1/Mean:0' shape=() dtype=float32>
}], metrics: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Cast_1:0' shape=(?,) dtype=float32>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'replica_1/Cast_1:0' shape=(?,) dtype=float32>
}]


...

task:default, lr:1.0000e-03, loss:0.0084, metric:1.0000:  15%|█▍        | 23/156 [00:00<00:00, 222.09it/s] 

'epoch 27 train time(sec): 0.6749556064605713'
{'best_train_metrics': [0.9891],
 'best_validate_metrics': [0.8261],
 'epoch_index': 27,
 'train_loss': [0.0322],
 'train_metrics': [0.9891],
 'validate_loss': [0.8283],
 'validate_metrics': [0.8228]}


task:default, lr:1.0000e-03, loss:0.0184, metric:1.0000:  14%|█▍        | 22/156 [00:00<00:00, 218.01it/s] 

'epoch 28 train time(sec): 0.6743841171264648'
{'best_train_metrics': [0.9905],
 'best_validate_metrics': [0.8261],
 'epoch_index': 28,
 'train_loss': [0.0329],
 'train_metrics': [0.9905],
 'validate_loss': [0.8290],
 'validate_metrics': [0.8243]}


'epoch 29 train time(sec): 0.6870405673980713'
{'best_train_metrics': [0.9905],
 'best_validate_metrics': [0.8261],
 'epoch_index': 29,
 'train_loss': [0.0309],
 'train_metrics': [0.9902],
 'validate_loss': [0.8447],
 'validate_metrics': [0.8235]}


## training with Adversarial Training

In [9]:
tf.compat.v1.reset_default_graph()
exec_model = TextCNN(config, word_dict=word_dict, name="cnn_model", distribute="tf_distribute")
adv_model = AdversarialTextClassWrapper(exec_model, config, word_dict=word_dict, 
                                        name="adv_model", distribute="tf_distribute")

2020-11-30 18:36:30,565  model.py (76) (MainThread) : INFO  config: {'unsup_coeff': 1, 'tsa_schedule': 'linear_schedule', 'batch_size': 32, 'unsup_ratio': 6, 'num_train_steps': 23400, 'use_adv_train': False, 'epsilon': 10, 'xi': 0.1, 'rampup_method': 'sigmoid', 'learning_rate': 0.001, 'num_classes': 2, 'max_seq_len': 100, 'embedding_size': 128, 'num_layers': 2, 'conv_num_filters': 256, 'keep_prob': 0.5, 'gpu_id': 0, 'gpu_num': 2, 'model_output_dir': './saved_models', 'model_name': 'cnn'}
2020-11-30 18:36:30,609  model.py (76) (MainThread) : INFO  config: {'unsup_coeff': 1, 'tsa_schedule': 'linear_schedule', 'batch_size': 32, 'unsup_ratio': 6, 'num_train_steps': 23400, 'use_adv_train': False, 'epsilon': 10, 'xi': 0.1, 'rampup_method': 'sigmoid', 'learning_rate': 0.001, 'num_classes': 2, 'max_seq_len': 100, 'embedding_size': 128, 'num_layers': 2, 'conv_num_filters': 256, 'keep_prob': 0.5, 'gpu_id': 0, 'gpu_num': 2, 'model_output_dir': './saved_models', 'model_name': 'cnn'}


In [10]:
adv_training = AdversarialTraining(adv_model, config, name='adv_training', distribute="tf_distribute")


Instructions for updating:
Use `tf.cast` instead.


2020-11-30 18:36:30,785  model.py (76) (MainThread) : INFO  config: {'unsup_coeff': 1, 'tsa_schedule': 'linear_schedule', 'batch_size': 32, 'unsup_ratio': 6, 'num_train_steps': 23400, 'use_adv_train': False, 'epsilon': 10, 'xi': 0.1, 'rampup_method': 'sigmoid', 'learning_rate': 0.001, 'num_classes': 2, 'max_seq_len': 100, 'embedding_size': 128, 'num_layers': 2, 'conv_num_filters': 256, 'keep_prob': 0.5, 'gpu_id': 0, 'gpu_num': 2, 'model_output_dir': './saved_models', 'model_name': 'cnn'}


In [11]:
adv_training.build_model()
print(adv_training.loss)
adv_training.model_summary()




Instructions for updating:
reduction_indices is deprecated, use axis instead
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead

use tsa schedule: linear_schedule
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Tensor("semi-sup-loss:0", shape=(), dtype=float32)
total parameter number: 581378


variable_name variable_shape parameters
0    AdvTrain/embedding_layer/embedding_table:0    [3765, 128]     481920
1  AdvTrain/my_conv_no.1/conv_1d/he_uniform/W:0  [1, 128, 128]      16384
2             AdvTrain/my_conv_no.1/conv_1d/B:0          [128]        128
3  AdvTrain/my_conv_no.2/conv_1d/he_uniform/W:0  [2, 128, 128]      32768
4             AdvTrain/my_conv_no.2/conv_1d/B:0          [128]        128
5  AdvTrain/my_conv_no.3/conv_1d/he_uniform/W:0  [3, 128, 128]      49152
6             AdvTrain/my_conv_no.3/conv_1d/B:0          [128]        128
7                  AdvTrain/fully_connected/W:0       [384, 2]        768
8                  AdvTrain/fully_connected/B:0            [2]          2

In [12]:
logging_hook = TensorInfoPrintHook({"unsup_loss": adv_training.unsup_loss,
                                    "epsilon": adv_training.epsilon,
#                                     "tsa_threshold": adv_training.tsa_threshold, 
                                    "sup_loss": adv_training.sup_loss})

In [13]:
train_data = SemiSupTrainData([sup_texts, org_unsup_texts, labels],
                     mode="train", batch_size=batch_size * config['gpu_num'], 
                     drop_remainder=True, name="train_data")

test_data = SemiSupTrainData([test_sup_texts, test_org_unsup_texts, test_labels],
                     mode="test", batch_size=batch_size * config['gpu_num'], 
                    drop_remainder=True, name="test_data")

2020-11-30 18:36:32,370  data.py (19) (MainThread) : INFO  train_data: will drop 16 remainder_size data items.
2020-11-30 18:36:32,654  data.py (19) (MainThread) : INFO  test_data: will drop 0 remainder_size data items.


In [14]:
adv_training.trainer.initialize()

2020-11-30 18:36:32,835  single_worker_graph_copy_trainer.py (92) (MainThread) : INFO  Initializing trainer ...
2020-11-30 18:36:32,835  single_worker_graph_copy_trainer.py (105) (MainThread) : INFO  gpu_num: 2
2020-11-30 18:36:32,837  graph_copy.py (424) (MainThread) : INFO  build_train_graph kwargs: {'unsup_coeff': 1, 'tsa_schedule': 'linear_schedule', 'batch_size': 32, 'unsup_ratio': 6, 'num_train_steps': 23400, 'use_adv_train': False, 'epsilon': 10, 'xi': 0.1, 'rampup_method': 'sigmoid', 'learning_rate': 0.001, 'num_classes': 2, 'max_seq_len': 100, 'embedding_size': 128, 'num_layers': 2, 'conv_num_filters': 256, 'keep_prob': 0.5, 'model_output_dir': './saved_models', 'model_name': 'cnn', 'excluded_task_set': ['pretrained_model/default'], 'pretrained_model_scope': 'pretrained_model'}
2020-11-30 18:36:32,837  graph_copy.py (426) (MainThread) : INFO  build_train_graph: reconstructing variables...



Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2020-11-30 18:36:34,291  graph_copy.py (448) (MainThread) : INFO  build_train_graph (adv_training): building inputs...
2020-11-30 18:36:34,292  graph_copy.py (450) (MainThread) : INFO  task: adv_training, <class 'str'>
2020-11-30 18:36:34,296  graph_copy.py (478) (MainThread) : INFO  build_train_graph (adv_training): building per replica graph...



INFO:tensorflow:batch_all_reduce: 9 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10


2020-11-30 18:36:36,357  graph_copy.py (486) (MainThread) : INFO  train_ops: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Adam_adv_training/Identity:0' shape=() dtype=int64>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'Adam_adv_training/Identity_1:0' shape=() dtype=int64>
}], losses: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'semi-sup-loss:0' shape=() dtype=float32>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'replica_1/semi-sup-loss:0' shape=() dtype=float32>
}], metrics: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Cast:0' shape=(?,) dtype=float32>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'replica_1/Cast:0' shape=(?,) dtype=float32>
}]


INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).


2020-11-30 18:36:36,391  single_worker_graph_copy_trainer.py (119) (MainThread) : INFO  all_tasks: ['adv_training']
2020-11-30 18:36:36,421  single_worker_graph_copy_trainer.py (122) (MainThread) : INFO  init variables with sess target: None
2020-11-30 18:36:37,239  single_worker_graph_copy_trainer.py (124) (MainThread) : INFO  single_worker: initialized
2020-11-30 18:36:37,240  single_worker_graph_copy_trainer.py (125) (MainThread) : INFO  Init time cost (secs):  4.404


In [15]:
adv_training.train_and_evaluate(train_data, test_data,
                batch_size=batch_size * config['gpu_num'],
                epochs=num_epochs,
                hooks=[logging_hook])

2020-11-30 18:36:37,245  single_worker_graph_copy_trainer.py (150) (MainThread) : INFO  tasks: ['adv_training']
2020-11-30 18:36:37,246  single_worker_graph_copy_trainer.py (151) (MainThread) : INFO  loss: Tensor("Identity:0", shape=(), dtype=float32, device=/replica:0/task:0/device:CPU:0), loss_list: [<tf.Tensor 'Identity:0' shape=() dtype=float32>], task name: adv_training
2020-11-30 18:36:37,246  single_worker_graph_copy_trainer.py (156) (MainThread) : INFO  do_test: True
2020-11-30 18:36:37,247  single_worker_graph_copy_trainer.py (157) (MainThread) : INFO  building dataset ...


sup_data_size: 9984, unsup_data_size: 66650


2020-11-30 18:36:37,539  single_worker_graph_copy_trainer.py (174) (MainThread) : INFO  start training sess run ...


last: {'unsup_loss': 0.03080955, 'epsilon': 0.07685659, 'sup_loss': 0.8737828}


task:adv_training, lr:1.0000e-03, loss:0.8073, metric:0.7812:   4%|▍         | 6/156 [00:00<00:02, 57.42it/s]

...

  0%|          | 0/156 [00:00<?, ?it/s]                                                                         

last: {'unsup_loss': 0.19189806, 'epsilon': 10.0, 'sup_loss': 0.2419111}
'epoch 147 train time(sec): 2.732574701309204'
{'best_train_metrics': [0.9952],
 'best_validate_metrics': [0.8477],
 'epoch_index': 147,
 'train_loss': [0.3786],
 'train_metrics': [0.9704],
 'validate_loss': [0.5993],
 'validate_metrics': [0.8416]}


  0%|          | 0/156 [00:00<?, ?it/s]                                                                         

last: {'unsup_loss': 0.28482738, 'epsilon': 10.0, 'sup_loss': 0.2117303}
'epoch 148 train time(sec): 2.7167859077453613'
{'best_train_metrics': [0.9952],
 'best_validate_metrics': [0.8477],
 'epoch_index': 148,
 'train_loss': [0.3660],
 'train_metrics': [0.9694],
 'validate_loss': [0.6174],
 'validate_metrics': [0.8412]}


last: {'unsup_loss': 0.1540042, 'epsilon': 10.0, 'sup_loss': 0.13562977}
'epoch 149 train time(sec): 2.7350270748138428'
{'best_train_metrics': [0.9952],
 'best_validate_metrics': [0.8477],
 'epoch_index': 149,
 'train_loss': [0.3611],
 'train_metrics': [0.9691],
 'validate_loss': [0.5606],
 'validate_metrics': [0.8417]}


In my experiment, with 10000 training data of `female_vulgar`, the validate accuracy improved from **0.8261** to **0.8498**.

## Export and infer

In [18]:
tf.compat.v1.reset_default_graph()
with tf.variable_scope('AdvTrain'):
    model = TextCNN(config, word_dict, name="default", distribute="tf_distribute")
    model.build_model()
print(model.model_summary())
model.trainer.initialize()
# note: change the ckpt path to your own path.
best_ckpt_path = 'saved_models/best_model/adv_training-epoch65.ckpt'
model.export_freeze_graph("freeze", 1, multi_replica_ckpt_path=best_ckpt_path)

2020-11-17 11:44:00,875  model.py (73) (MainThread) : INFO  config: {'unsup_coeff': 1, 'tsa_schedule': 'linear_schedule', 'batch_size': 32, 'unsup_ratio': 6, 'num_train_steps': 23200, 'use_adv_train': False, 'epsilon': 8, 'xi': 0.1, 'learning_rate': 0.001, 'num_classes': 2, 'max_seq_len': 100, 'embedding_size': 128, 'num_layers': 2, 'conv_num_filters': 256, 'keep_prob': 0.4, 'gpu_id': 0, 'gpu_num': 2, 'model_output_dir': './saved_models', 'model_name': 'cnn'}
2020-11-17 11:44:01,273  single_worker_graph_copy_trainer.py (92) (MainThread) : INFO  Initializing trainer ...
2020-11-17 11:44:01,274  single_worker_graph_copy_trainer.py (105) (MainThread) : INFO  gpu_num: 2
2020-11-17 11:44:01,275  graph_copy.py (424) (MainThread) : INFO  build_train_graph kwargs: {'unsup_coeff': 1, 'tsa_schedule': 'linear_schedule', 'batch_size': 32, 'unsup_ratio': 6, 'num_train_steps': 23200, 'use_adv_train': False, 'epsilon': 8, 'xi': 0.1, 'learning_rate': 0.001, 'num_classes': 2, 'max_seq_len': 100, 'embed

total parameter number: 581378
                                  variable_name variable_shape parameters
0    AdvTrain/embedding_layer/embedding_table:0    [3765, 128]     481920
1  AdvTrain/my_conv_no.1/conv_1d/he_uniform/W:0  [1, 128, 128]      16384
2             AdvTrain/my_conv_no.1/conv_1d/B:0          [128]        128
3  AdvTrain/my_conv_no.2/conv_1d/he_uniform/W:0  [2, 128, 128]      32768
4             AdvTrain/my_conv_no.2/conv_1d/B:0          [128]        128
5  AdvTrain/my_conv_no.3/conv_1d/he_uniform/W:0  [3, 128, 128]      49152
6             AdvTrain/my_conv_no.3/conv_1d/B:0          [128]        128
7                  AdvTrain/fully_connected/W:0       [384, 2]        768
8                  AdvTrain/fully_connected/B:0            [2]          2


2020-11-17 11:44:01,886  graph_copy.py (448) (MainThread) : INFO  build_train_graph (default): building inputs...
2020-11-17 11:44:01,886  graph_copy.py (450) (MainThread) : INFO  task: default, <class 'str'>
2020-11-17 11:44:01,891  graph_copy.py (478) (MainThread) : INFO  build_train_graph (default): building per replica graph...


INFO:tensorflow:batch_all_reduce: 9 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10


2020-11-17 11:44:02,730  graph_copy.py (486) (MainThread) : INFO  train_ops: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Adam_default/Identity:0' shape=() dtype=int64>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'Adam_default/Identity_1:0' shape=() dtype=int64>
}], losses: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'AdvTrain/Mean:0' shape=() dtype=float32>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'replica_1/AdvTrain/Mean:0' shape=() dtype=float32>
}], metrics: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'AdvTrain/Cast_1:0' shape=(?,) dtype=float32>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'replica_1/AdvTrain/Cast_1:0' shape=(?,) dtype=float32>
}]


INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).


2020-11-17 11:44:02,892  single_worker_graph_copy_trainer.py (119) (MainThread) : INFO  all_tasks: ['default']
2020-11-17 11:44:02,930  single_worker_graph_copy_trainer.py (122) (MainThread) : INFO  init variables with sess target: None
2020-11-17 11:44:03,150  single_worker_graph_copy_trainer.py (124) (MainThread) : INFO  single_worker: initialized
2020-11-17 11:44:03,150  single_worker_graph_copy_trainer.py (125) (MainThread) : INFO  Init time cost (secs):  1.876
2020-11-17 11:44:03,152  single_worker_graph_copy_trainer.py (431) (MainThread) : INFO  Restore 9 vars from checkpoint.



Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 9 variables.
INFO:tensorflow:Converted 9 variables to const ops.


In [19]:
def load_feeze_graph(model_fn):
    gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
    config = tf.compat.v1.ConfigProto(gpu_options=gpu_options, allow_soft_placement=True)
    session = tf.compat.v1.Session(config=config, graph=tf.Graph())
    with session.graph.as_default():
        tf.compat.v1.train.import_meta_graph(model_fn)
        session.run(tf.compat.v1.tables_initializer())
    return session

sess = load_feeze_graph('freeze/1/frozen.pb')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [17]:
with sess.graph.as_default():
    inputs = tf.get_collection('input_dict')
    outputs = tf.get_collection('output_dict')[0]

    data_iter = DataIterator([test_sup_texts], 
                             np.array(range(len(test_sup_texts))), batch_size=batch_size)
    data_iter_tqdm = tqdm(data_iter, leave=False)
    res = []
    for idx, batch in data_iter_tqdm:
        feed_data = dict(zip(inputs, batch))
        batch_res = sess.run(outputs, feed_dict=feed_data)
        res.append(batch_res)
       
    np_res = np.concatenate(res, axis=0)
    
print(np_res)

/root/anaconda3/envs/tf1.15/lib/python3.6/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


[[0.21800725 0.78199273]
 [0.24221507 0.75778496]
 [0.39926568 0.60073435]
 ...
 [0.6999417  0.3000583 ]
 [0.44201118 0.5579889 ]
 [0.7353714  0.26462856]]


In [18]:
pos_predicts = np.argmax(np_res, axis=1)
pos_acc = accuracy_score(test_labels, pos_predicts)
print(f"acc on test: {pos_acc}")

acc on test: 0.8498
